# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,20.86,96,97,1.34,EC,1730335777
1,1,port elizabeth,-33.9180,25.5701,16.29,88,75,2.06,ZA,1730335779
2,2,port alfred,-33.5906,26.8910,12.88,96,5,3.65,ZA,1730335780
3,3,port-aux-francais,-49.3500,70.2167,4.25,85,32,12.17,TF,1730335781
4,4,tiksi,71.6872,128.8694,-24.06,100,77,1.42,RU,1730335782


In [7]:
print(city_data_df.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='City', cmap='Category20', size='Humidity', alpha=0.6,
    tiles='CartoLight', hover_cols=['City', 'Country', 'Humidity'], width=1000, height=500
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,geraldton,-28.7667,114.6000,25.23,47,0,3.09,AU,1730335808
53,53,hamilton,39.1834,-84.5333,21.65,62,0,2.68,US,1730335838
107,107,aiken,33.5835,-81.5998,22.01,77,0,2.57,US,1730335903
129,129,remire-montjoly,4.9167,-52.2667,26.02,89,0,0.00,GF,1730335930
144,144,malvan,16.0667,73.4667,25.45,86,0,1.91,IN,1730335949


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
27,geraldton,AU,-28.7667,114.6000,47,
53,hamilton,US,39.1834,-84.5333,62,
107,aiken,US,33.5835,-81.5998,77,
129,remire-montjoly,GF,4.9167,-52.2667,89,
144,malvan,IN,16.0667,73.4667,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set search radius
radius = 10000  

# Base URL for Geoapify
base_url = "https://api.geoapify.com/v2/places"

# Print start message
print("Starting hotel search")

# Iterate through hotel_df to find the nearest hotel for each city
for index, row in hotel_df.iterrows():
    # Set parameters for API request
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{row['Lng']},{row['Lat']},{radius}",
        "bias": f"proximity:{row['Lng']},{row['Lat']}",
        "limit": 1,
        "apiKey": geoapify_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Store hotel name or indicate if not found
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log search result
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
geraldton - nearest hotel: Ocean Centre Hotel
hamilton - nearest hotel: North Vista Manor
aiken - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
malvan - nearest hotel: Hotel Silver Sand
baise city - nearest hotel: 百色宾馆
ruiz - nearest hotel: No hotel found
izki - nearest hotel: Cloudy Cloud Farm Resort
ellettsville - nearest hotel: Quality Inn
ribas do rio pardo - nearest hotel: Hotel São Francisco
fortaleza - nearest hotel: Hotel Catedral
natchez - nearest hotel: Natchez Grand Hotel
merta - nearest hotel: No hotel found
ballina - nearest hotel: Ballina Manor Boutique Hotel
mangrol - nearest hotel: No hotel found
carbondale - nearest hotel: Home2 Suites by Hilton Carbondale


,City,Country,Lat,Lng,Humidity,Hotel Name
27,geraldton,AU,-28.7667,114.6000,47,Ocean Centre Hotel
53,hamilton,US,39.1834,-84.5333,62,North Vista Manor
107,aiken,US,33.5835,-81.5998,77,No hotel found
129,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
144,malvan,IN,16.0667,73.4667,86,Hotel Silver Sand


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='City', cmap='Category20', size='Humidity', alpha=0.8,
    tiles='OSM', hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    width=1000, height=600
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)